<a href="https://colab.research.google.com/github/NatasaKinzy/Data-Science/blob/main/Unit7/NataKinzUnit7ExercisesSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install preliz
!pip install bambi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.2/529.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.6/164.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
      Successfully uninstalled numba-0.60.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.9/218.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 

In [2]:
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
import preliz as pz
import pymc as pm
import bambi as bmb

# Fitting Curves: Concepts

What you'll do:

- Answer questions about what a GP is, and its relationship to GLMs and splines.
- Practice applying each of: polynomial modeling, b splines, and GPs
- You'll get a chance to read about and try to comprehend a more standard implementation of a GP.

Have fun!

**Task1**:

Why would you ever want to include a polynomial element in a model you built? What's the benefit of using polynomials to model?

**Polynomial elements add more degrees to the model and allow your model to fit observed data better. If you know all your possible inputs (there's no input your data doesn't consider) adding more and more degrees will usually simply fit your data better. If there is any section of the graph that doesn't follow the main trend adding degrees can allow you to create a curvy model that fits your dataset more accurately.**

**Task2**:

Why would you ever NOT want to include a polynomial element in a model you built?

**The higher the degree you use the more at risk you are for overfitting your model. When data has a simpler relationship or many inputs your dataset doesn't consider, polynomial elements are very much at risk of overfitting. The simplest model you can have is the ideal, so if you data can be modeled by a linear, exponential, or logarithmic relationship, you dont need to try to make it more complicated than it is.**

**Task3**:

What's the point of using b splines?

**B splines can give you curves with less risk of overfitting. B splines slice your x axis into parts based on your degree and fit each part as best they can. You can also weight specific concentrations or clumps in the dataset so the b splines fit even better.**

**Task4**:

Describe what a Gaussian Process is, in your own words. *Don't worry about being correct, just try to explain it to yourself*. I will not grade this question for accuracy.

**Gausian processes relate all possible data points to other data points and find patterns between data rather than show the overall trend of the data. They provide more flexibility but run very slowly.**

**Task5**:

Fit three models to the howell data (from Unit5ExercisesSF): polynomial, b splines, and Gaussian Process.

Plot the posterior predictive check on a scatter plot, as is standard/required.

Hint: Distributional models (variable variance) work better on the howell data.


In [3]:
howell = pd.read_csv('https://raw.githubusercontent.com/thedarredondo/data-science-fundamentals/main/Data/howell.csv')

In [4]:
howell

,height,weight,age,male
0,151.765,47.825606,63.0,1
1,139.700,36.485807,63.0,0
2,136.525,31.864838,65.0,0
3,156.845,53.041914,41.0,1
4,145.415,41.276872,51.0,0
...,...,...,...,...
539,145.415,31.127751,17.0,1
540,162.560,52.163080,31.0,1
541,156.210,54.062497,21.0,0
542,71.120,8.051258,0.0,1


In [ ]:
#Polynomial
model_poly_4 = bmb.Model("rented ~ poly(hour, degree=4)", bikes, family="negativebinomial")
idata_poly_4 = model_poly_4.fit()

In [ ]:
bmb.interpret.plot_predictions(model_poly_4, idata_poly_4, "hour", pps=True)

In [ ]:
plt.plot(bikes.hour, bikes.rented, "C2.", zorder=-3)

In [ ]:
#B Spline
num_knots = 6
knots = np.linspace(0, 23, num_knots+2)[1:-1]
model_spline = bmb.Model("rented ~ bs(hour, degree=3, knots=knots)", bikes, family="negativebinomial")
idata_spline = model_spline.fit()

In [ ]:
bmb.interpret.plot_predictions(model_spline, idata_spline, "hour", pps= True)
plt.plot(bikes.hour, bikes.rented, "C2.", zorder=-3)

In [ ]:
#GP
rented = bikes["rented"].values
hour = bikes["hour"].values[:,None]

In [ ]:
def get_ig_params(x_vals, l_b=None, u_b=None, mass=0.96, plot=False):
    """
    Returns a weakly informative prior for the length-scale parameter of the GP kernel.
    """

    differences = np.abs(np.subtract.outer(x_vals, x_vals))
    if l_b is None:
        l_b = np.min(differences[differences != 0]) * 2
    if u_b is None:
        u_b = np.max(differences) / 1.5

    dist = pz.InverseGamma()
    pz.maxent(dist, l_b, u_b, mass, plot=plot)

    return dict(zip(dist.param_names, dist.params))

In [ ]:
prior_gp = {
    "sigma": bmb.Prior("Gamma", mu=1, sigma=0.01),
    "ell": bmb.Prior("InverseGamma", **get_ig_params(hour))
}

priors = {
    "hsgp(hour, m=168, c=8.0)": prior_gp,
    "alpha": bmb.Prior("HalfNormal", sigma=1)
}

model_hsb = bmb.Model("rented ~ 0 + hsgp(hour, m=168, c=8.0)", bikes, family="negativebinomial", priors=priors)
idata_hsb = model_hsb.fit()

In [ ]:
_, ax = bmb.interpret.plot_predictions(model_hsb, idata_hsb, ["hour"],
                                       pps = True,
                                       fig_kwargs={"figsize": (10, 3)})
ax[0].plot(bikes["hour"].values, bikes["rented"].values, "C2.")

**Task6**:

Read the article on the pymc website about GP implementation on the Mauna Loa CO$_{2}$ data combined with CO$_{2}$ ice core data from the south pole.
[Link here.](https://www.pymc.io/projects/examples/en/latest/gaussian_processes/GP-MaunaLoa2.html)

Write down one thing you learned about GPs from reading the article.

Note: You probably won't understand much in this article--I had to read it about five times before I figured out what was going on. The points of this task are to: hammer home that good GP implementations are extremely technical, and increasure your exposure to the kinds of problems traditional GPs are good at solving.

**When you assign a GP a mean, it will stick to that mean once the observed dataset has no more information (as in while it's predicting some of the infinite possibilities without data on them), but that may not always align with patterns we're seeing in the real world. I wonder if you can combat that by adding degrees to a polynomial in the model or if you just need to input new data to get the GP to understand what's happenning.**

**Task7**:

Describe your favorite graph from the article in the previous task with as much technical detail as you can muster.

Explain why its your favorite.

**My favorite model is the CO2 vs year model under predictions. It looks like an hourglass turned on its side, stretched out in the middle and wiggled around. Its blue, black, and white. There's an initial max at 295 and minimum at 263. It's then a bit straight and then becomes a concave down quadratic (a hump) at year 150-275. Then it goes fairly straight (imagine your hand is shaking while drawing a straight line) with a tiny slope upward. Then at year 1000-1350 there's another concave down hump that's fairly flat. Then there's a much steeper concave down hump from 1400-1600, and the graph slopes down and then waves up again at 1600ish. Then it becomes exponential from 1650-1900, timed with industrial revolution. Then it reaches new maximum of 333 and mininum 286 around 2050. Then it ends with the bottom of the hourglass with maximum of 324 and minimum of 275 around 2200 which hasn't happened yet so it's just predicting our future.**

**It's my favorite because I've never seen a model that has that shape. It's also icy looking while modeling ice ages in a way. And I think it's really amazing how well you can see the CO2 spike during the industrial revolution through modeling CO2 stored in ice. It's just crazy to me that we can aquire data in that way.**